In [59]:
import seaborn as sns
import math as math
from random import gauss as gs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
import itertools
from pmdarima import auto_arima
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import roc_curve, roc_auc_score, auc

#statsmodels
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from keras.preprocessing.sequence import TimeseriesGenerator
from pathlib import Path
import os


In [56]:
def CleanNegatives(df,Tag,CleanName):
    InBlock = False  # Tags that I am inside of a -999 block
    Start = 0
    End   = 0
    Clean = df[Tag]
    for Index in range(len(Clean)):
        if InBlock:
            if Clean[Index] > 0: # Exit condition 
                End = Index
                InBlock = False
                if Start ==0:
                    AVG = Clean[End]
                elif End == len(Clean) - 1:
                    AVG = Clean[Start]
                else:
                    AVG = (Clean[Start-1]+Clean[End])/2
                # Cleaning negatives, replacing by average at the endpoints of the block
                for NewIndex in range(Start,End):
                    Clean[NewIndex] = AVG
        else:  # Not In the block
            if Clean[Index] < 0: # Enter condition
                Start = Index
                InBlock = True
    df[CleanName] = Clean
                

In [3]:
def Cleanup(df,Tag1,Tag2,ResultTag):
    Clean = []
    for i in range(len(df)):
        Value = ( df[Tag1][i] if not math.isnan(df[Tag1][i]) else df[Tag2][i])
        Clean.append(Value)
    df[ResultTag] = Clean

In [4]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [5]:
datapath = Path.cwd() / "data"
print(datapath)

/Users/dc/Desktop/Flatiron/projects/Capstone-Disney-World-Date-and-Attendance-Predictor/data


In [6]:
datasets = os.listdir(datapath)
datasets = list(sorted(datasets))
print(datasets)

['7_dwarfs_train.csv', 'dinosaur.csv', 'entities.csv', 'expedition_everest.csv', 'kilimanjaro_safaris.csv', 'metadata.csv', 'navi_river.csv', 'pirates_of_caribbean.csv', 'rock_n_rollercoaster.csv', 'slinky_dog.csv', 'soarin.csv', 'spaceship_earth.csv', 'splash_mountain.csv', 'touringplans_data_dictionary.xlsx', 'toy_story_mania.csv']


In [7]:
for i in datasets:
    try:
        exec(f"{i.split('.csv')[0]} = pd.read_csv(datapath / i)")
    except:
        print(f"{i} cannot be imported automatically")

7_dwarfs_train.csv cannot be imported automatically
touringplans_data_dictionary.xlsx cannot be imported automatically


In [8]:
seven_dwarfs_train = pd.read_csv(datapath / "7_dwarfs_train.csv")
datasets[0] = "seven_dwarfs_train.csv"

In [9]:
for i in datasets:
    try:
        print(i)
        display(eval(f"{i.split('.csv')[0]}.head()"), eval(f"len({i.split('.csv')[0]})"))
    except:
        pass

seven_dwarfs_train.csv


,date,datetime,SACTMIN,SPOSTMIN
0,01/01/2015,2015-01-01 07:51:12,NaN,45.0
1,01/01/2015,2015-01-01 08:02:13,NaN,60.0
2,01/01/2015,2015-01-01 08:05:30,54.0,NaN
3,01/01/2015,2015-01-01 08:09:12,NaN,60.0
4,01/01/2015,2015-01-01 08:16:12,NaN,60.0


321631

dinosaur.csv


,date,datetime,SACTMIN,SPOSTMIN
0,01/01/2015,2015-01-01 07:47:26,NaN,5.0
1,01/01/2015,2015-01-01 07:54:23,NaN,5.0
2,01/01/2015,2015-01-01 08:05:33,NaN,5.0
3,01/01/2015,2015-01-01 08:12:23,NaN,10.0
4,01/01/2015,2015-01-01 08:19:26,NaN,10.0


...

,date,datetime,SACTMIN,SPOSTMIN
0,01/01/2015,2015-01-01 07:51:12,NaN,5.0
1,01/01/2015,2015-01-01 08:02:13,NaN,5.0
2,01/01/2015,2015-01-01 08:09:12,NaN,5.0
3,01/01/2015,2015-01-01 08:16:12,NaN,5.0
4,01/01/2015,2015-01-01 08:23:12,NaN,5.0


287948

touringplans_data_dictionary.xlsx
toy_story_mania.csv


,date,datetime,SACTMIN,SPOSTMIN
0,01/01/2015,2015-01-01 07:51:16,NaN,20.0
1,01/01/2015,2015-01-01 07:53:19,NaN,20.0
2,01/01/2015,2015-01-01 08:04:29,NaN,30.0
3,01/01/2015,2015-01-01 08:11:19,NaN,30.0
4,01/01/2015,2015-01-01 08:12:31,NaN,40.0


284170

In [10]:
print(sorted(dinosaur["SACTMIN"].unique()))

[nan, 0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0, 76.0, 77.0, 78.0, 79.0, 81.0, 82.0, 83.0, 84.0, 86.0, 89.0, 91.0, 94.0, 95.0, 96.0]


In [11]:
print(sorted(dinosaur["SPOSTMIN"].unique()))

[-999.0, 0.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0, 40.0, 45.0, 50.0, 55.0, 60.0, 65.0, 70.0, 80.0, nan, 75.0, 85.0, 90.0, 95.0, 100.0, 105.0, 110.0, 115.0, 120.0, 125.0, 130.0, 135.0, 140.0, 145.0, 150.0, 155.0, 165.0, 170.0, 175.0, 180.0, 255.0, 300.0]


In [12]:
print(dinosaur.dtypes)

date         object
datetime     object
SACTMIN     float64
SPOSTMIN    float64
dtype: object


In [13]:
dinosaur_data = dinosaur.merge(metadata, how= "left", left_on= "date", right_on= "DATE")
display(dinosaur_data.head())
print(len(dinosaur_data))

,date,datetime,SACTMIN,SPOSTMIN,DATE,WDW_TICKET_SEASON,DAYOFWEEK,DAYOFYEAR,WEEKOFYEAR,MONTHOFYEAR,YEAR,SEASON,HOLIDAYPX,HOLIDAYM,HOLIDAYN,HOLIDAY,WDWTICKETSEASON,WDWRaceN,WDWeventN,WDWevent,WDWrace,WDWSEASON,WDWMAXTEMP,WDWMINTEMP,WDWMEANTEMP,MKeventN,MKevent,EPeventN,EPevent,HSeventN,HSevent,AKeventN,AKevent,HOLIDAYJ,inSession,inSession_Enrollment,inSession_wdw,inSession_dlr,inSession_sqrt_WDW,inSession_sqrt_DLR,inSession_California,inSession_DC,inSession_Central_FL,inSession_Drive1_FL,inSession_Drive2_FL,inSession_Drive_CA,inSession_Florida,inSession_Mardi_Gras,inSession_Midwest,inSession_NY_NJ,inSession_NY_NJ_PA,inSession_New_England,inSession_New_Jersey,inSession_Nothwest,INSESSION_PLANES,inSession_SoCal,inSession_Southwest,MKEMHMORN,MKEMHMYEST,MKEMHMTOM,MKEMHEVE,MKHOURSEMH,MKHOURSEMHYEST,MKHOURSEMHTOM,MKEMHEYEST,MKEMHETOM,EPEMHMORN,EPEMHMYEST,EPEMHMTOM,EPEMHEVE,EPEMHEYEST,EPEMHETOM,EPHOURSEMH,EPHOURSEMHYEST,EPHOURSEMHTOM,HSEMHMORN,HSEMHMYEST,HSEMHMTOM,HSEMHEVE,HSEMHEYEST,HSEMHETOM,HSHOURSEMH,HSHOURSEMHYEST,HSHOURSEMHTOM,AKEMHMORN,AKEMHMYEST,AKEMHMTOM,AKEMHEVE,AKEMHEYEST,AKEMHETOM,AKHOURSEMH,AKHOURSEMHYEST,AKHOURSEMHTOM,MKOPEN,MKCLOSE,MKHOURS,MKEMHOPEN,MKEMHCLOSE,MKOPENYEST,MKCLOSEYEST,MKHOURSYEST,MKOPENTOM,MKCLOSETOM,MKHOURSTOM,EPOPEN,EPCLOSE,EPHOURS,EPEMHOPEN,EPEMHCLOSE,EPOPENYEST,EPCLOSEYEST,EPHOURSYEST,EPOPENTOM,EPCLOSETOM,EPHOURSTOM,HSOPEN,HSCLOSE,HSHOURS,HSEMHOPEN,HSEMHCLOSE,HSOPENYEST,HSCLOSEYEST,HSHOURSYEST,HSOPENTOM,HSCLOSETOM,HSHOURSTOM,AKOPEN,AKCLOSE,AKHOURS,AKEMHOPEN,AKEMHCLOSE,AKOPENYEST,AKCLOSEYEST,AKHOURSYEST,AKOPENTOM,AKCLOSETOM,AKHOURSTOM,WEATHER_WDWHIGH,WEATHER_WDWLOW,WEATHER_WDWPRECIP,CapacityLost_MK,CapacityLost_EP,CapacityLost_HS,CapacityLost_AK,CapacityLostWGT_MK,CapacityLostWGT_EP,CapacityLostWGT_HS,CapacityLostWGT_AK,MKPRDDAY,MKPRDDT1,MKPRDDT2,MKPRDDN,MKPRDNGT,MKPRDNT1,MKPRDNT2,MKPRDNN,MKFIREWK,MKFIRET1,MKFIRET2,MKFIREN,EPFIREWK,EPFIRET1,EPFIRET2,EPFIREN,HSPRDDAY,HSPRDDT1,HSPRDDN,HSFIREWK,HSFIRET1,HSFIRET2,HSFIREN,HSSHWNGT,HSSHWNT1,HSSHWNT2,HSSHWNN,HSFIREWKS,AKPRDDAY,AKPRDDT1,AKPRDDT2,AKPRDDN,AKFIREN,AKSHWNGT,AKSHWNT1,AKSHWNT2,AKSHWNN
0,01/01/2015,2015-01-01 07:47:26,NaN,5.0,01/01/2015,NaN,5.0,0.0,0.0,1.0,2015.0,CHRISTMAS PEAK,0.0,5.0,nyd,1.0,NaN,NaN,NaN,0.0,0.0,CHRISTMAS PEAK,73.02,59.81,66.41,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,1.0,1.0,0.0,0.0,18.0,19.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,13.0,17.0,15.0,0.0,0.0,0.0,0.0,1.0,0.0,14.0,16.0,14.0,0.0,1.0,0.0,0.0,0.0,0.0,11.0,15.0,12.0,8:00,25:00,17.0,7:00,25:00,8:00,26:00,18.0,8:00,25:00,17.0,8:00,21:00,13.0,8:00,21:00,8:00,25:00,17.0,8:00,21:00,13.0,8:00,22:00,14.0,8:00,22:00,8:00,24:00,16.0,8:00,22:00,14.0,8:00,19:00,11.0,8:00,19:00,8:00,22:00,14.0,8:00,20:00,12.0,70.3,50.2,0.12,352865.0,325168.0,203780.0,210779.0,34661635.0,31692832.0,19812520.0,20790321.0,2.0,12:00,15:30,Disney Festival of Fantasy Parade,1.0,22:15,NaN,Main Street Electrical Parade,1.0,21:00,NaN,Wishes Nighttime Spectacular,1.0,21:00,NaN,IllumiNations: Reflections of Earth,0.0,NaN,NaN,0.0,NaN,NaN,NaN,3.0,18:30,20:00,Fantasmic!,1.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
1,01/01/2015,2015-01-01 07:54:23,NaN,5.0,01/01/2015,NaN,5.0,0.0,0.0,1.0,2015.0,CHRISTMAS PEAK,0.0,5.0,nyd,1.0,NaN,NaN,NaN,0.0,0.0,CHRISTMAS PEAK,73.02,59.81,66.41,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,1.0,1.0,0.0,0.0,18.0,19.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,13.0,17.0,15.0,0.0,0.0,0.0,0.0,1.0,0.0,14.0,16.0,14.0,0.0,1.0,0.0,0.0,0.0,0.0,11.0,15.0,12.0,8:00,25:00,17.0,7:00,25:00,8:00,26:00,18.0,8:00,25:00,17.0,8:00,21:00,13.0,8:00,21:00,8:00,25:00,17.0,8:00,21:00,13.0,8:00,22:00,14.0,8:00,22:00,8:00,24:00,16.0,8:00,22:00,14.0,8:00,19:00,11.0,8:00,19:00,8:00,22:00,14.0,8:00,20:00,12.0,70.3,50.2,0.12,352865.0,325168.0,203780.0,210779.0,34661635.0,31692832.0,19812520.0,20790321.0,2.0,12:00,15:30,Disney Festival of Fantasy Parade,1.0,22:15,NaN,Main Street Electrical Parade,1.0,21:00,NaN,Wi

252403


In [14]:
CleanNegatives(dinosaur_data,'SPOSTMIN','SPOSTMINClean')
print(dinosaur_data)

<ipython-input-2-a4f54c8aedc8>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Clean[NewIndex] = AVG
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [15]:
Cleanup(dinosaur_data,'SACTMIN','SPOSTMINClean','CleanTime')
print(dinosaur_data)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [16]:
dinosaur_data.head()

,date,datetime,SACTMIN,SPOSTMIN,DATE,WDW_TICKET_SEASON,DAYOFWEEK,DAYOFYEAR,WEEKOFYEAR,MONTHOFYEAR,YEAR,SEASON,HOLIDAYPX,HOLIDAYM,HOLIDAYN,HOLIDAY,WDWTICKETSEASON,WDWRaceN,WDWeventN,WDWevent,WDWrace,WDWSEASON,WDWMAXTEMP,WDWMINTEMP,WDWMEANTEMP,MKeventN,MKevent,EPeventN,EPevent,HSeventN,HSevent,AKeventN,AKevent,HOLIDAYJ,inSession,inSession_Enrollment,inSession_wdw,inSession_dlr,inSession_sqrt_WDW,inSession_sqrt_DLR,inSession_California,inSession_DC,inSession_Central_FL,inSession_Drive1_FL,inSession_Drive2_FL,inSession_Drive_CA,inSession_Florida,inSession_Mardi_Gras,inSession_Midwest,inSession_NY_NJ,inSession_NY_NJ_PA,inSession_New_England,inSession_New_Jersey,inSession_Nothwest,INSESSION_PLANES,inSession_SoCal,inSession_Southwest,MKEMHMORN,MKEMHMYEST,MKEMHMTOM,MKEMHEVE,MKHOURSEMH,MKHOURSEMHYEST,MKHOURSEMHTOM,MKEMHEYEST,MKEMHETOM,EPEMHMORN,EPEMHMYEST,EPEMHMTOM,EPEMHEVE,EPEMHEYEST,EPEMHETOM,EPHOURSEMH,EPHOURSEMHYEST,EPHOURSEMHTOM,HSEMHMORN,HSEMHMYEST,HSEMHMTOM,HSEMHEVE,HSEMHEYEST,HSEMHETOM,HSHOURSEMH,HSHOURSEMHYEST,HSHOURSEMHTOM,AKEMHMORN,AKEMHMYEST,AKEMHMTOM,AKEMHEVE,AKEMHEYEST,AKEMHETOM,AKHOURSEMH,AKHOURSEMHYEST,AKHOURSEMHTOM,MKOPEN,MKCLOSE,MKHOURS,MKEMHOPEN,MKEMHCLOSE,MKOPENYEST,MKCLOSEYEST,MKHOURSYEST,MKOPENTOM,MKCLOSETOM,MKHOURSTOM,EPOPEN,EPCLOSE,EPHOURS,EPEMHOPEN,EPEMHCLOSE,EPOPENYEST,EPCLOSEYEST,EPHOURSYEST,EPOPENTOM,EPCLOSETOM,EPHOURSTOM,HSOPEN,HSCLOSE,HSHOURS,HSEMHOPEN,HSEMHCLOSE,HSOPENYEST,HSCLOSEYEST,HSHOURSYEST,HSOPENTOM,HSCLOSETOM,HSHOURSTOM,AKOPEN,AKCLOSE,AKHOURS,AKEMHOPEN,AKEMHCLOSE,AKOPENYEST,AKCLOSEYEST,AKHOURSYEST,AKOPENTOM,AKCLOSETOM,AKHOURSTOM,WEATHER_WDWHIGH,WEATHER_WDWLOW,WEATHER_WDWPRECIP,CapacityLost_MK,CapacityLost_EP,CapacityLost_HS,CapacityLost_AK,CapacityLostWGT_MK,CapacityLostWGT_EP,CapacityLostWGT_HS,CapacityLostWGT_AK,MKPRDDAY,MKPRDDT1,MKPRDDT2,MKPRDDN,MKPRDNGT,MKPRDNT1,MKPRDNT2,MKPRDNN,MKFIREWK,MKFIRET1,MKFIRET2,MKFIREN,EPFIREWK,EPFIRET1,EPFIRET2,EPFIREN,HSPRDDAY,HSPRDDT1,HSPRDDN,HSFIREWK,HSFIRET1,HSFIRET2,HSFIREN,HSSHWNGT,HSSHWNT1,HSSHWNT2,HSSHWNN,HSFIREWKS,AKPRDDAY,AKPRDDT1,AKPRDDT2,AKPRDDN,AKFIREN,AKSHWNGT,AKSHWNT1,AKSHWNT2,AKSHWNN,SPOSTMINClean,CleanTime
0,01/01/2015,2015-01-01 07:47:26,NaN,5.0,01/01/2015,NaN,5.0,0.0,0.0,1.0,2015.0,CHRISTMAS PEAK,0.0,5.0,nyd,1.0,NaN,NaN,NaN,0.0,0.0,CHRISTMAS PEAK,73.02,59.81,66.41,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,1.0,1.0,0.0,0.0,18.0,19.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,13.0,17.0,15.0,0.0,0.0,0.0,0.0,1.0,0.0,14.0,16.0,14.0,0.0,1.0,0.0,0.0,0.0,0.0,11.0,15.0,12.0,8:00,25:00,17.0,7:00,25:00,8:00,26:00,18.0,8:00,25:00,17.0,8:00,21:00,13.0,8:00,21:00,8:00,25:00,17.0,8:00,21:00,13.0,8:00,22:00,14.0,8:00,22:00,8:00,24:00,16.0,8:00,22:00,14.0,8:00,19:00,11.0,8:00,19:00,8:00,22:00,14.0,8:00,20:00,12.0,70.3,50.2,0.12,352865.0,325168.0,203780.0,210779.0,34661635.0,31692832.0,19812520.0,20790321.0,2.0,12:00,15:30,Disney Festival of Fantasy Parade,1.0,22:15,NaN,Main Street Electrical Parade,1.0,21:00,NaN,Wishes Nighttime Spectacular,1.0,21:00,NaN,IllumiNations: Reflections of Earth,0.0,NaN,NaN,0.0,NaN,NaN,NaN,3.0,18:30,20:00,Fantasmic!,1.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,5.0,5.0
1,01/01/2015,2015-01-01 07:54:23,NaN,5.0,01/01/2015,NaN,5.0,0.0,0.0,1.0,2015.0,CHRISTMAS PEAK,0.0,5.0,nyd,1.0,NaN,NaN,NaN,0.0,0.0,CHRISTMAS PEAK,73.02,59.81,66.41,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,1.0,1.0,0.0,0.0,18.0,19.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,13.0,17.0,15.0,0.0,0.0,0.0,0.0,1.0,0.0,14.0,16.0,14.0,0.0,1.0,0.0,0.0,0.0,0.0,11.0,15.0,12.0,8:00,25:00,17.0,7:00,25:00,8:00,26:00,18.0,8:00,25:00,17.0,8:00,21:00,13.0,8:00,21:00,8:00,25:00,17.0,8:00,21:00,13.0,8:00,22:00,14.0,8:00,22:00,8:00,24:00,16.0,8:00,22:00,14.0,8:00,19:00,11.0,8:00,19:00,8:00,22:00,14.0,8:00,20:00,12.0,70.3,50.2,0.12,352865.0,325168.0,203780.0,210779.0,34661635.0,31692832.0,19812520.0,20790321.0,2.0,12:00,15:30,Disney Festival of Fantasy Parade,1.0,22:15,NaN,Main Street El

In [17]:
dinosaur_data.fillna(-1, inplace=True)
display(dinosaur_data.head())

,date,datetime,SACTMIN,SPOSTMIN,DATE,WDW_TICKET_SEASON,DAYOFWEEK,DAYOFYEAR,WEEKOFYEAR,MONTHOFYEAR,YEAR,SEASON,HOLIDAYPX,HOLIDAYM,HOLIDAYN,HOLIDAY,WDWTICKETSEASON,WDWRaceN,WDWeventN,WDWevent,WDWrace,WDWSEASON,WDWMAXTEMP,WDWMINTEMP,WDWMEANTEMP,MKeventN,MKevent,EPeventN,EPevent,HSeventN,HSevent,AKeventN,AKevent,HOLIDAYJ,inSession,inSession_Enrollment,inSession_wdw,inSession_dlr,inSession_sqrt_WDW,inSession_sqrt_DLR,inSession_California,inSession_DC,inSession_Central_FL,inSession_Drive1_FL,inSession_Drive2_FL,inSession_Drive_CA,inSession_Florida,inSession_Mardi_Gras,inSession_Midwest,inSession_NY_NJ,inSession_NY_NJ_PA,inSession_New_England,inSession_New_Jersey,inSession_Nothwest,INSESSION_PLANES,inSession_SoCal,inSession_Southwest,MKEMHMORN,MKEMHMYEST,MKEMHMTOM,MKEMHEVE,MKHOURSEMH,MKHOURSEMHYEST,MKHOURSEMHTOM,MKEMHEYEST,MKEMHETOM,EPEMHMORN,EPEMHMYEST,EPEMHMTOM,EPEMHEVE,EPEMHEYEST,EPEMHETOM,EPHOURSEMH,EPHOURSEMHYEST,EPHOURSEMHTOM,HSEMHMORN,HSEMHMYEST,HSEMHMTOM,HSEMHEVE,HSEMHEYEST,HSEMHETOM,HSHOURSEMH,HSHOURSEMHYEST,HSHOURSEMHTOM,AKEMHMORN,AKEMHMYEST,AKEMHMTOM,AKEMHEVE,AKEMHEYEST,AKEMHETOM,AKHOURSEMH,AKHOURSEMHYEST,AKHOURSEMHTOM,MKOPEN,MKCLOSE,MKHOURS,MKEMHOPEN,MKEMHCLOSE,MKOPENYEST,MKCLOSEYEST,MKHOURSYEST,MKOPENTOM,MKCLOSETOM,MKHOURSTOM,EPOPEN,EPCLOSE,EPHOURS,EPEMHOPEN,EPEMHCLOSE,EPOPENYEST,EPCLOSEYEST,EPHOURSYEST,EPOPENTOM,EPCLOSETOM,EPHOURSTOM,HSOPEN,HSCLOSE,HSHOURS,HSEMHOPEN,HSEMHCLOSE,HSOPENYEST,HSCLOSEYEST,HSHOURSYEST,HSOPENTOM,HSCLOSETOM,HSHOURSTOM,AKOPEN,AKCLOSE,AKHOURS,AKEMHOPEN,AKEMHCLOSE,AKOPENYEST,AKCLOSEYEST,AKHOURSYEST,AKOPENTOM,AKCLOSETOM,AKHOURSTOM,WEATHER_WDWHIGH,WEATHER_WDWLOW,WEATHER_WDWPRECIP,CapacityLost_MK,CapacityLost_EP,CapacityLost_HS,CapacityLost_AK,CapacityLostWGT_MK,CapacityLostWGT_EP,CapacityLostWGT_HS,CapacityLostWGT_AK,MKPRDDAY,MKPRDDT1,MKPRDDT2,MKPRDDN,MKPRDNGT,MKPRDNT1,MKPRDNT2,MKPRDNN,MKFIREWK,MKFIRET1,MKFIRET2,MKFIREN,EPFIREWK,EPFIRET1,EPFIRET2,EPFIREN,HSPRDDAY,HSPRDDT1,HSPRDDN,HSFIREWK,HSFIRET1,HSFIRET2,HSFIREN,HSSHWNGT,HSSHWNT1,HSSHWNT2,HSSHWNN,HSFIREWKS,AKPRDDAY,AKPRDDT1,AKPRDDT2,AKPRDDN,AKFIREN,AKSHWNGT,AKSHWNT1,AKSHWNT2,AKSHWNN,SPOSTMINClean,CleanTime
0,01/01/2015,2015-01-01 07:47:26,-1.0,5.0,01/01/2015,-1,5.0,0.0,0.0,1.0,2015.0,CHRISTMAS PEAK,0.0,5.0,nyd,1.0,-1,-1,-1,0.0,0.0,CHRISTMAS PEAK,73.02,59.81,66.41,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,1.0,1.0,0.0,0.0,18.0,19.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,13.0,17.0,15.0,0.0,0.0,0.0,0.0,1.0,0.0,14.0,16.0,14.0,0.0,1.0,0.0,0.0,0.0,0.0,11.0,15.0,12.0,8:00,25:00,17.0,7:00,25:00,8:00,26:00,18.0,8:00,25:00,17.0,8:00,21:00,13.0,8:00,21:00,8:00,25:00,17.0,8:00,21:00,13.0,8:00,22:00,14.0,8:00,22:00,8:00,24:00,16.0,8:00,22:00,14.0,8:00,19:00,11.0,8:00,19:00,8:00,22:00,14.0,8:00,20:00,12.0,70.3,50.2,0.12,352865.0,325168.0,203780.0,210779.0,34661635.0,31692832.0,19812520.0,20790321.0,2.0,12:00,15:30,Disney Festival of Fantasy Parade,1.0,22:15,-1,Main Street Electrical Parade,1.0,21:00,-1,Wishes Nighttime Spectacular,1.0,21:00,-1,IllumiNations: Reflections of Earth,0.0,-1.0,-1.0,0.0,-1,-1,-1,3.0,18:30,20:00,Fantasmic!,1.0,0.0,-1.0,-1.0,-1.0,-1.0,0.0,-1,-1,-1,5.0,5.0
1,01/01/2015,2015-01-01 07:54:23,-1.0,5.0,01/01/2015,-1,5.0,0.0,0.0,1.0,2015.0,CHRISTMAS PEAK,0.0,5.0,nyd,1.0,-1,-1,-1,0.0,0.0,CHRISTMAS PEAK,73.02,59.81,66.41,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,1.0,1.0,0.0,0.0,18.0,19.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,13.0,17.0,15.0,0.0,0.0,0.0,0.0,1.0,0.0,14.0,16.0,14.0,0.0,1.0,0.0,0.0,0.0,0.0,11.0,15.0,12.0,8:00,25:00,17.0,7:00,25:00,8:00,26:00,18.0,8:00,25:00,17.0,8:00,21:00,13.0,8:00,21:00,8:00,25:00,17.0,8:00,21:00,13.0,8:00,22:00,14.0,8:00,22:00,8:00,24:00,16.0,8:00,22:00,14.0,8:00,19:00,11.0,8:00,19:00,8:00,22:00,14.0,8:00,20:00,12.0,70.3,50.2,0.12,352865.0,325168.0,203780.0,210779.0,34661635.0,31692832.0,19812520.0,20790321.0,2.0,12:00,15:30,Disney Festival of Fantasy Parade,1.0,22:15,-1,Main Street Electrical Parade,1.0,

In [18]:
dinosaur_data.drop('SACTMIN', axis=1, inplace=True)

# dinosaur_data.drop(columns = "SACTMIN", "SPOSTMIN", "DATE", "SPOSTMINClean")

In [19]:
dinosaur_data.drop('SPOSTMIN', axis=1, inplace=True)

In [20]:
dinosaur_data.drop('SPOSTMINClean', axis=1, inplace=True)

In [21]:
dtypes = dinosaur_data.dtypes
print(dtypes)

date                      object
datetime                  object
DATE                      object
WDW_TICKET_SEASON         object
DAYOFWEEK                float64
DAYOFYEAR                float64
WEEKOFYEAR               float64
MONTHOFYEAR              float64
YEAR                     float64
SEASON                    object
HOLIDAYPX                float64
HOLIDAYM                 float64
HOLIDAYN                  object
HOLIDAY                  float64
WDWTICKETSEASON           object
WDWRaceN                  object
WDWeventN                 object
WDWevent                 float64
WDWrace                  float64
WDWSEASON                 object
WDWMAXTEMP               float64
WDWMINTEMP               float64
WDWMEANTEMP              float64
MKeventN                  object
MKevent                  float64
EPeventN                  object
EPevent                  float64
HSeventN                  object
HSevent                  float64
AKeventN                  object
AKevent   

In [22]:
print(str(dinosaur_data.dtypes["date"]))
cols = []
for i, j in enumerate(dtypes):
    if str(j) != "float64":
        cols.append(dinosaur_data.dtypes.index[i])
print(cols)

object
['date', 'datetime', 'DATE', 'WDW_TICKET_SEASON', 'SEASON', 'HOLIDAYN', 'WDWTICKETSEASON', 'WDWRaceN', 'WDWeventN', 'WDWSEASON', 'MKeventN', 'EPeventN', 'HSeventN', 'AKeventN', 'HOLIDAYJ', 'inSession', 'inSession_Enrollment', 'inSession_wdw', 'inSession_dlr', 'inSession_sqrt_WDW', 'inSession_sqrt_DLR', 'inSession_California', 'inSession_DC', 'inSession_Central_FL', 'inSession_Drive1_FL', 'inSession_Drive2_FL', 'inSession_Drive_CA', 'inSession_Florida', 'inSession_Mardi_Gras', 'inSession_Midwest', 'inSession_NY_NJ', 'inSession_NY_NJ_PA', 'inSession_New_England', 'inSession_New_Jersey', 'inSession_Nothwest', 'INSESSION_PLANES', 'inSession_SoCal', 'inSession_Southwest', 'MKOPEN', 'MKCLOSE', 'MKEMHOPEN', 'MKEMHCLOSE', 'MKOPENYEST', 'MKCLOSEYEST', 'MKOPENTOM', 'MKCLOSETOM', 'EPOPEN', 'EPCLOSE', 'EPEMHOPEN', 'EPEMHCLOSE', 'EPOPENYEST', 'EPCLOSEYEST', 'EPOPENTOM', 'EPCLOSETOM', 'HSOPEN', 'HSCLOSE', 'HSEMHOPEN', 'HSEMHCLOSE', 'HSOPENYEST', 'HSCLOSEYEST', 'HSOPENTOM', 'HSCLOSETOM', 'AKOP

In [23]:
def encode_labels(col):
    le = LabelEncoder()
    le.fit(col)
    col = le.transform(col)
    return col, le

In [24]:
for i in range(3, len(cols)):
    dinosaur_data[cols[i]] = dinosaur_data[cols[i]].astype(str)
    dinosaur_data[cols[i]], _ = encode_labels(dinosaur_data[cols[i]])

In [25]:
display(dinosaur_data.head())

,date,datetime,DATE,WDW_TICKET_SEASON,DAYOFWEEK,DAYOFYEAR,WEEKOFYEAR,MONTHOFYEAR,YEAR,SEASON,HOLIDAYPX,HOLIDAYM,HOLIDAYN,HOLIDAY,WDWTICKETSEASON,WDWRaceN,WDWeventN,WDWevent,WDWrace,WDWSEASON,WDWMAXTEMP,WDWMINTEMP,WDWMEANTEMP,MKeventN,MKevent,EPeventN,EPevent,HSeventN,HSevent,AKeventN,AKevent,HOLIDAYJ,inSession,inSession_Enrollment,inSession_wdw,inSession_dlr,inSession_sqrt_WDW,inSession_sqrt_DLR,inSession_California,inSession_DC,inSession_Central_FL,inSession_Drive1_FL,inSession_Drive2_FL,inSession_Drive_CA,inSession_Florida,inSession_Mardi_Gras,inSession_Midwest,inSession_NY_NJ,inSession_NY_NJ_PA,inSession_New_England,inSession_New_Jersey,inSession_Nothwest,INSESSION_PLANES,inSession_SoCal,inSession_Southwest,MKEMHMORN,MKEMHMYEST,MKEMHMTOM,MKEMHEVE,MKHOURSEMH,MKHOURSEMHYEST,MKHOURSEMHTOM,MKEMHEYEST,MKEMHETOM,EPEMHMORN,EPEMHMYEST,EPEMHMTOM,EPEMHEVE,EPEMHEYEST,EPEMHETOM,EPHOURSEMH,EPHOURSEMHYEST,EPHOURSEMHTOM,HSEMHMORN,HSEMHMYEST,HSEMHMTOM,HSEMHEVE,HSEMHEYEST,HSEMHETOM,HSHOURSEMH,HSHOURSEMHYEST,HSHOURSEMHTOM,AKEMHMORN,AKEMHMYEST,AKEMHMTOM,AKEMHEVE,AKEMHEYEST,AKEMHETOM,AKHOURSEMH,AKHOURSEMHYEST,AKHOURSEMHTOM,MKOPEN,MKCLOSE,MKHOURS,MKEMHOPEN,MKEMHCLOSE,MKOPENYEST,MKCLOSEYEST,MKHOURSYEST,MKOPENTOM,MKCLOSETOM,MKHOURSTOM,EPOPEN,EPCLOSE,EPHOURS,EPEMHOPEN,EPEMHCLOSE,EPOPENYEST,EPCLOSEYEST,EPHOURSYEST,EPOPENTOM,EPCLOSETOM,EPHOURSTOM,HSOPEN,HSCLOSE,HSHOURS,HSEMHOPEN,HSEMHCLOSE,HSOPENYEST,HSCLOSEYEST,HSHOURSYEST,HSOPENTOM,HSCLOSETOM,HSHOURSTOM,AKOPEN,AKCLOSE,AKHOURS,AKEMHOPEN,AKEMHCLOSE,AKOPENYEST,AKCLOSEYEST,AKHOURSYEST,AKOPENTOM,AKCLOSETOM,AKHOURSTOM,WEATHER_WDWHIGH,WEATHER_WDWLOW,WEATHER_WDWPRECIP,CapacityLost_MK,CapacityLost_EP,CapacityLost_HS,CapacityLost_AK,CapacityLostWGT_MK,CapacityLostWGT_EP,CapacityLostWGT_HS,CapacityLostWGT_AK,MKPRDDAY,MKPRDDT1,MKPRDDT2,MKPRDDN,MKPRDNGT,MKPRDNT1,MKPRDNT2,MKPRDNN,MKFIREWK,MKFIRET1,MKFIRET2,MKFIREN,EPFIREWK,EPFIRET1,EPFIRET2,EPFIREN,HSPRDDAY,HSPRDDT1,HSPRDDN,HSFIREWK,HSFIRET1,HSFIRET2,HSFIREN,HSSHWNGT,HSSHWNT1,HSSHWNT2,HSSHWNN,HSFIREWKS,AKPRDDAY,AKPRDDT1,AKPRDDT2,AKPRDDN,AKFIREN,AKSHWNGT,AKSHWNT1,AKSHWNT2,AKSHWNN,CleanTime
0,01/01/2015,2015-01-01 07:47:26,01/01/2015,0,5.0,0.0,0.0,1.0,2015.0,2,0.0,5.0,26,1.0,0,0,0,0.0,0.0,2,73.02,59.81,66.41,0,0.0,0,0.0,0,0.0,0,0.0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1.0,1.0,0.0,0.0,18.0,19.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,13.0,17.0,15.0,0.0,0.0,0.0,0.0,1.0,0.0,14.0,16.0,14.0,0.0,1.0,0.0,0.0,0.0,0.0,11.0,15.0,12.0,3,10,17.0,2,10,3,11,18.0,3,10,17.0,5,3,13.0,5,3,5,8,17.0,5,3,13.0,4,10,14.0,5,10,4,13,16.0,4,10,14.0,2,7,11.0,2,7,2,13,14.0,2,9,12.0,70.3,50.2,0.12,352865.0,325168.0,203780.0,210779.0,34661635.0,31692832.0,19812520.0,20790321.0,2.0,2,3,1,1.0,9,0,1,1.0,8,0,8,1.0,3,0,2,0.0,-1.0,-1.0,0.0,0,0,0,3.0,2,1,1,1.0,0.0,-1.0,-1.0,-1.0,-1.0,0.0,0,0,0,5.0
1,01/01/2015,2015-01-01 07:54:23,01/01/2015,0,5.0,0.0,0.0,1.0,2015.0,2,0.0,5.0,26,1.0,0,0,0,0.0,0.0,2,73.02,59.81,66.41,0,0.0,0,0.0,0,0.0,0,0.0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1.0,1.0,0.0,0.0,18.0,19.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,13.0,17.0,15.0,0.0,0.0,0.0,0.0,1.0,0.0,14.0,16.0,14.0,0.0,1.0,0.0,0.0,0.0,0.0,11.0,15.0,12.0,3,10,17.0,2,10,3,11,18.0,3,10,17.0,5,3,13.0,5,3,5,8,17.0,5,3,13.0,4,10,14.0,5,10,4,13,16.0,4,10,14.0,2,7,11.0,2,7,2,13,14.0,2,9,12.0,70.3,50.2,0.12,352865.0,325168.0,203780.0,210779.0,34661635.0,31692832.0,19812520.0,20790321.0,2.0,2,3,1,1.0,9,0,1,1.0,8,0,8,1.0,3,0,2,0.0,-1.0,-1.0,0.0,0,0,0,3.0,2,1,1,1.0,0.0,-1.0,-1.0,-1.0,-1.0,0.0,0,0,0,5.0
2,01/01/2015,2015-01-01 08:05:33,01/01/2015,0,5.0,0.0,0.0,1.0,2015.0,2,0.0,5.0,26,1.0,0,0,0,0.0,0.0,2,73.02,59.81,66.41,0,0.0,0,0.0,0,0.0,0,0.0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1.0,1.0,0.0,0.0,18.0,19.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,13.0,17.0,15.0,0.0,0.0,0.0,0.0,1.0,0.0,14.0,16.0,14.0,0.0,1.0,0.0,0.0,0.0,0.0,11.0,15.0,12.0,3,10,17.0,2,10,3,11,18.0,3,10,17.0,5,3,13.0,5,3,5,8,17.0,5,3,13.0,4,10,14.0,5,10,4,13,16.0,4,10,14.0,2,7,11.0,2,7,2,13,14.0,2,9,12.0,70.3,50.2,0.12,352865.0,325168.0,203780.0,21

In [26]:
X = dinosaur_data.drop(columns = ["date", "datetime", "DATE"])
y = dinosaur_data["CleanTime"]

In [27]:
from sklearn.linear_model import Perceptron

clf = Perceptron(tol=1e-3, random_state=0)
clf.fit(X, y)
print(clf.score(X, y))


ValueError: Unknown label type: (array([ -1. ,   0. ,   1. ,   2. ,   2.5,   3. ,   4. ,   5. ,   6. ,
         7. ,   7.5,   8. ,   9. ,  10. ,  11. ,  12. ,  12.5,  13. ,
        14. ,  15. ,  16. ,  17. ,  17.5,  18. ,  19. ,  20. ,  21. ,
        22. ,  22.5,  23. ,  24. ,  25. ,  26. ,  27. ,  27.5,  28. ,
        29. ,  30. ,  31. ,  32. ,  32.5,  33. ,  34. ,  35. ,  36. ,
        37. ,  37.5,  38. ,  39. ,  40. ,  41. ,  42. ,  42.5,  43. ,
        44. ,  45. ,  46. ,  47. ,  47.5,  48. ,  49. ,  50. ,  51. ,
        52. ,  52.5,  53. ,  54. ,  55. ,  56. ,  57. ,  57.5,  58. ,
        59. ,  60. ,  61. ,  62. ,  62.5,  63. ,  64. ,  65. ,  66. ,
        67. ,  67.5,  68. ,  69. ,  70. ,  71. ,  72. ,  72.5,  73. ,
        74. ,  75. ,  76. ,  77. ,  77.5,  78. ,  79. ,  80. ,  81. ,
        82. ,  82.5,  83. ,  84. ,  85. ,  86. ,  89. ,  90. ,  91. ,
        92.5,  94. ,  95. ,  96. , 100. , 102.5, 105. , 107.5, 110. ,
       115. , 120. , 125. , 130. , 135. , 140. , 145. , 1

In [41]:
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

In [42]:
X = dinosaur_data.drop(columns = ["date", "datetime", "DATE", "CleanTime"])
y = dinosaur_data["CleanTime"]

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

regr = MLPRegressor(random_state=1, max_iter=500).fit(X_train, y_train)
regr.predict(X_test)
                                                    
regr.score(X_test, y_test)

-463.9468051768737

### Random Forest Model

#### Train Model

In [51]:
X = dinosaur_data.drop(columns = ["date", "datetime", "DATE", "CleanTime"], axis=1)
y = dinosaur_data["CleanTime"]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [63]:
rf_model = RandomForestRegressor(n_estimators = 500)
rf_model.fit(X_train,y_train)

RandomForestRegressor(n_estimators=500)

In [64]:
y_test_pred=rf_model.predict(X_test)
test_accuracy=(y_test ==y_test_pred).sum() / len(y_test)
print('Test Set accuracy: ',test_accuracy)

Test Set accuracy:  0.003597407331104103
